In [1]:
library(tidyverse)
library(RPostgreSQL)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.0 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: DBI


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "sql")
list.files(sql_dir)

[1] "basics.sql"              "cohort.sql"             
[3] "icd9.sql"                "lab_tests.sql"          
[5] "lab_unpivot.sql"         "population.sql"         
[7] "vital_signs.sql"         "vital_signs_unpivot.sql"

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
labs <- "select distinct(label) from lab_tests" %>%
    dbGetQuery(con, .) %>%
    pull(label)
labs

[1] "hemoglobin"        "wbc"               "lactate"          
 [4] "po2"               "troponin"          "potassium"        
 [7] "creatinine_kinase" "creatinine"        "pco2"             
[10] "bnp"               "bun"               "bicarbonate"      
[13] "platelet"          "sodium"            "chloride"         
[16] "ph"

In [6]:
sql_template <- "
with lab_summary as (
    select distinct hadm_id, label
    , first_value(valuenum) over (partition by hadm_id, label order by charttime) as fst_val
    , first_value(valuenum) over (partition by hadm_id, label order by valuenum) as min_val
    , first_value(valuenum) over (partition by hadm_id, label order by valuenum desc) as max_val
    from lab_tests
)

select hadm_id
, %s
from lab_summary
group by hadm_id
"

In [7]:
sql <- c("max(case when label = '%1$s' then 1 else 0 end) as lab_%1$s_flag",
  "max(case when label = '%1$s' then fst_val else null end) as lab_%1$s_first",
  "max(case when label = '%1$s' then min_val else null end) as lab_%1$s_min",
  "max(case when label = '%1$s' then max_val else null end) as lab_%1$s_max") %>%
paste(collapse = "\n, ") %>%
sprintf(labs) %>%
paste(collapse = "\n, ") %>%
sprintf(sql_template, .)
cat(sql, file = file.path(sql_dir, "lab_unpivot.sql"))

In [8]:
sql %>% make_view_sql("lab_unpivot", "materialized view") %>% dbSendQuery(con, .)

<PostgreSQLResult>

In [9]:
dbGetQuery(con, "select * from lab_unpivot limit 10")

hadm_id,lab_hemoglobin_flag,lab_hemoglobin_first,lab_hemoglobin_min,lab_hemoglobin_max,lab_wbc_flag,lab_wbc_first,lab_wbc_min,lab_wbc_max,lab_lactate_flag,⋯,lab_sodium_min,lab_sodium_max,lab_chloride_flag,lab_chloride_first,lab_chloride_min,lab_chloride_max,lab_ph_flag,lab_ph_first,lab_ph_min,lab_ph_max
125078,1,13.0,13.0,14.4,1,13.6,13.6,16.9,1,⋯,160,165,1,123,123,128,1,7.45,7.42,7.45
151232,1,9.8,9.8,9.8,1,7.8,7.8,7.8,0,⋯,139,139,1,105,105,105,0,NA,NA,NA
164444,1,15.0,15.0,15.0,1,12.9,12.9,12.9,0,⋯,142,146,1,108,107,112,1,7.49,7.49,7.49
146726,1,12.4,12.4,12.4,1,7.2,7.2,7.2,1,⋯,134,136,1,100,100,105,1,7.44,7.44,7.44
160170,1,11.9,11.9,11.9,1,2.1,2.1,2.1,0,⋯,138,138,1,103,103,103,0,NA,NA,NA
112553,1,8.3,8.3,8.3,1,17.5,17.5,17.5,0,⋯,143,147,1,118,113,118,0,NA,NA,NA
151997,1,7.8,7.8,7.8,1,3.4,3.4,3.4,1,⋯,131,148,1,97,97,120,1,7.61,7.43,7.61
158611,1,10.4,10.4,10.4,1,17.0,17.0,17.0,0,⋯,129,129,1,98,98,98,0,NA,NA,NA
105158,1,12.3,8.8,12.3,1,61.5,61.5,76.9,1,⋯,146,151,1,108,104,108,1,6.82,6.82,7.30
194870,1,11.8,11.3,11.8,1,20.3,15.3,20.3,1,⋯,135,136,1,99,99,104,1,7.09,7.09,7.33


In [10]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE